## Importando Dataset - Campeonato Brasileiro

In [71]:
import pandas as pd
import numpy as np

csv_path = '.\dataset\campeonato-brasileiro-full.csv'

df = pd.read_csv(csv_path, delimiter=';')

## Capturando informações gerais das colunas

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8440 entries, 0 to 8439
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                8440 non-null   int64  
 1   Rodada            8440 non-null   object 
 2   Data              8436 non-null   object 
 3   Horário           8079 non-null   object 
 4   Dia               8436 non-null   object 
 5   Mandante          8440 non-null   object 
 6   Visitante         8440 non-null   object 
 7   Vencedor          8440 non-null   object 
 8   Arena             8436 non-null   object 
 9   Mandante Placar   8436 non-null   float64
 10  Visitante Placar  8436 non-null   float64
 11  Estado Mandante   8440 non-null   object 
 12  Estado Visitante  8440 non-null   object 
 13  Estado Vencedor   8440 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 923.2+ KB


## Listando os 5 primeiros registros

In [73]:
df.head(5)

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor
0,1,1,2000-07-29,16h00,Sábado,Fluminense,Bahia,Fluminense,Maracanã,2.0,0.0,RJ,BA,RJ
1,2,1,2000-07-29,16h00,Sábado,Vasco,Sport,Sport,São Januário,0.0,2.0,RJ,PE,PE
2,3,1,2000-07-29,16h00,Sábado,Vitória,Palmeiras,Vitória,Barradão,4.0,1.0,ES,SP,ES
3,4,1,2000-07-30,17h00,Domingo,Botafogo-RJ,Atlético-MG,-,Caio Martins,0.0,0.0,RJ,MG,-
4,5,1,2000-07-30,18h30,Domingo,Juventude,Flamengo,-,Alfredo Jaconi,1.0,1.0,RS,RJ,-


## Tratando dados inválidos

In [74]:
from unicodedata import normalize

def trataString(value):
    return normalize('NFKD', str(value)).encode('ASCII', 'ignore').decode('ASCII').upper()

df['Mandante'] = df['Mandante'].apply(trataString)
df['Visitante'] = df['Visitante'].apply(trataString)
df['Dia'] = df['Dia'].apply(trataString)
df['Vencedor'] = df['Vencedor'].apply(trataString)
df['Arena'] = df['Arena'].apply(trataString)


## Ordenando o Dataset pelo Mandante da partida

In [87]:
df['Pontos Mandante'] = 0
df['Pontos Visitante'] = 0

df.loc[ df['Vencedor'] == df['Mandante'], 'Pontos Mandante' ] = 3
df.loc[ df['Vencedor'] == df['Visitante'], 'Pontos Visitante' ] = 3
df.loc[ df['Vencedor'] == '-', 'Pontos Mandante' ] = 1
df.loc[ df['Vencedor'] == '-', 'Pontos Visitante' ] = 1


times = pd.Series(df['Mandante'].unique(), name='Times')
tg = pd.DataFrame(times)
tg['Pontos'] = 0

test = df.groupby('Mandante')['Pontos Mandante'].sum()

for i in test.index:
    i

## Ordenando o Dataset pelas maiores goleadas

In [76]:
df['Diferenca Gols'] = abs(df['Mandante Placar'] - df['Visitante Placar'])

# dropando valores nulos, visto que se não houve o jogo, provavelmente foi remarcado e está em outro registro
df = df.dropna(subset=['Diferenca Gols'])

df.sort_values(by=['Diferenca Gols'], ascending=False)

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Pontos Mandante,Pontos Visitante,Diferenca Gols
1600,1601,46,2003-12-14,16h00,DOMINGO,BAHIA,CRUZEIRO,CRUZEIRO,FONTE NOVA,0.0,7.0,BA,MG,MG,0,3,7.0
1122,1123,6,2003-04-27,16h00,DOMINGO,GOIAS,JUVENTUDE,GOIAS,SERRA DOURADA,7.0,0.0,GO,RS,GO,3,0,7.0
1992,1993,33,2004-09-28,20h30,TERCA-FEIRA,SAO PAULO,PAYSANDU,SAO PAULO,MORUMBI,7.0,0.0,SP,PA,SP,3,0,7.0
1528,1529,40,2003-11-02,16h00,DOMINGO,FIGUEIRENSE,PAYSANDU,FIGUEIRENSE,ORLANDO SCARPELLI,6.0,0.0,SC,PA,SC,3,0,6.0
1338,1339,24,2003-08-07,20h30,QUINTA-FEIRA,FLAMENGO,BAHIA,FLAMENGO,MARACANA,6.0,0.0,RJ,BA,RJ,3,0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740,2741,13,2006-07-22,18h10,SABADO,FLUMINENSE,SAO CAETANO,-,MARACANA,2.0,2.0,RJ,SP,-,1,1,0.0
3750,3751,38,2008-12-07,17h00,DOMINGO,SANTOS,NAUTICO,-,VILA BELMIRO,0.0,0.0,SP,PE,-,1,1,0.0
1666,1667,6,2004-05-16,16h00,DOMINGO,JUVENTUDE,ATLETICO-MG,-,ALFREDO JACONI,1.0,1.0,RS,MG,-,1,1,0.0
6932,6933,14,2017-07-16,16:00,DOMINGO,VASCO,SANTOS,-,ENGENHAO,0.0,0.0,RJ,SP,-,1,1,0.0


## Exibir as 5 maiores contagens de valores únicos de uma única coluna
#### No caso: Quem são os melhores Mandantes ?

In [77]:
df[ df['Mandante'] == df['Vencedor'] ][['Mandante']].value_counts().head()

Mandante     
SANTOS           233
SAO PAULO        231
GREMIO           223
ATHLETICO-PR     221
INTERNACIONAL    217
dtype: int64

## Exiba, no mínimo, três plots. Se possível, use um para variáveis quantitativa e outro para variáveis de qualitativas

In [78]:
# Implementar

## Implemente pelo menos um filtro

In [79]:
# Implementar

## Utilize, no mínimo, uma função agregada com agrupamento

In [80]:
df.groupby(by=['Estado Mandante'])['Mandante Placar'].sum() + df.groupby(by=['Estado Visitante'])['Visitante Placar'].sum()

Estado Mandante
AL      24.0
BA     521.0
CE     477.0
DF     141.0
ES     615.0
GO    1061.0
MG    2438.0
MT      12.0
PA     229.0
PE     880.0
PR    2147.0
RJ    3969.0
RN      24.0
RS    2426.0
SC    1269.0
SP    6011.0
dtype: float64

In [81]:
df

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Pontos Mandante,Pontos Visitante,Diferenca Gols
0,1,1,2000-07-29,16h00,SABADO,FLUMINENSE,BAHIA,FLUMINENSE,MARACANA,2.0,0.0,RJ,BA,RJ,3,0,2.0
1,2,1,2000-07-29,16h00,SABADO,VASCO,SPORT,SPORT,SAO JANUARIO,0.0,2.0,RJ,PE,PE,0,3,2.0
2,3,1,2000-07-29,16h00,SABADO,VITORIA,PALMEIRAS,VITORIA,BARRADAO,4.0,1.0,ES,SP,ES,3,0,3.0
3,4,1,2000-07-30,17h00,DOMINGO,BOTAFOGO-RJ,ATLETICO-MG,-,CAIO MARTINS,0.0,0.0,RJ,MG,-,1,1,0.0
4,5,1,2000-07-30,18h30,DOMINGO,JUVENTUDE,FLAMENGO,-,ALFREDO JACONI,1.0,1.0,RS,RJ,-,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435,8436,12,2021-07-18,16:00,DOMINGO,ATLETICO-GO,PALMEIRAS,PALMEIRAS,ANTONIO ACCIOLY,0.0,3.0,GO,SP,SP,0,3,3.0
8436,8437,12,2021-07-18,18:15,DOMINGO,BAHIA,FLAMENGO,FLAMENGO,PITUACU,0.0,5.0,BA,RJ,RJ,0,3,5.0
8437,8438,12,2021-07-18,20:30,DOMINGO,INTERNACIONAL,JUVENTUDE,INTERNACIONAL,BEIRA-RIO,1.0,0.0,RS,RS,RS,3,0,1.0
8438,8439,12,2021-07-18,20:30,DOMINGO,BRAGANTINO,SANTOS,-,NABI ABI CHEDID,2.0,2.0,SP,SP,-,1,1,0.0


In [82]:
df['Mandante'].unique()

array(['FLUMINENSE', 'VASCO', 'VITORIA', 'BOTAFOGO-RJ', 'JUVENTUDE',
       'GUARANI', 'CRUZEIRO', 'AMERICA-MG', 'GOIAS', 'CORITIBA', 'BAHIA',
       'FLAMENGO', 'GAMA', 'INTERNACIONAL', 'PONTE PRETA', 'SANTOS',
       'PALMEIRAS', 'ATHLETICO-PR', 'CORINTHIANS', 'ATLETICO-MG',
       'PORTUGUESA', 'GREMIO', 'SAO PAULO', 'SANTA CRUZ', 'SPORT',
       'PARANA', 'REMO', 'SAO CAETANO', 'MALUTROM', 'BOTAFOGO-SP',
       'FIGUEIRENSE', 'PAYSANDU', 'FORTALEZA', 'CRICIUMA', 'BRASILIENSE',
       'AMERICA-RN', 'NAUTICO', 'IPATINGA', 'AVAI', 'SANTO ANDRE',
       'BARUERI', 'ATLETICO-GO', 'CEARA', 'GREMIO PRUDENTE',
       'CHAPECOENSE', 'JOINVILLE', 'CSA', 'BRAGANTINO', 'CUIABA'],
      dtype=object)